In [1]:
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(palette='summer')

In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00


In [4]:
!pip install transformers

In [5]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [6]:
import transformers
from datasets import load_dataset
import evaluate

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

In [8]:
billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [9]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

In [10]:
billsum = billsum.train_test_split(test_size=0.1)

In [11]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1113
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 124
    })
})

In [12]:
tokenizer = transformers.AutoTokenizer.from_pretrained("ainize/bart-base-cnn")

tokenizer_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [13]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["text"], max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/1113 [00:00<?, ? examples/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

In [15]:
tokenized_billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1113
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 124
    })
})

In [16]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained("ainize/bart-base-cnn")

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [18]:
# Функция которая отдельно собирает примеры в батчи
## Принимает по сути inputs / label и делает из них квадратные батчи, чтобы подавать в модель

data_collator = transformers.DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [19]:
# аргументы для тренировки

training_args = transformers.Seq2SeqTrainingArguments(
        output_dir="./results", # путь куда будут сохраняться сопотствующие файлы
        evaluation_strategy="epoch", # оценка качества (каждую эпоху, каждое n количество шагов и др)
        learning_rate=2e-5, # шаг обучения
        per_device_train_batch_size=4, # размер батча для тренировки
        per_device_eval_batch_size=4, # размер батча для оценки
        weight_decay=0.01, # для оптимизатора
        save_total_limit=3, # то, сколько чекпоинтов храним (в случае ограничения памяти удаляем ненужные уже)
        num_train_epochs=2, # количество эпох
    )

In [20]:
trainer = transformers.Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.955095
2,2.183100,1.889978


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


TrainOutput(global_step=558, training_loss=2.1646345541895933, metrics={'train_runtime': 402.2405, 'train_samples_per_second': 5.534, 'train_steps_per_second': 1.387, 'total_flos': 1357273356042240.0, 'train_loss': 2.1646345541895933, 'epoch': 2.0})

###  Предсказания на тестовом множестве

In [25]:
text_example = billsum["test"]['text'][0]
print(text_example)

The people of the State of California do enact as follows:


SECTION 1.
It is the intent of the Legislature that this act shall not be construed to limit any of the following:
(a) The authority of the Legislature to create and fund new judgeships pursuant to Section 4 of Article VI of the California Constitution.
(b) The authority of the Governor to appoint a person to fill a vacancy pursuant to subdivision (c) of Section 16 of Article VI of the California Constitution.
(c) The authority of the Chief Justice of California to assign judges pursuant to subdivision (e) of Section 6 of Article VI of the California Constitution.
SEC. 2.
Section 69614.5 is added to the Government Code, to read:
69614.5.
(a) To provide for a more equitable distribution of judgeships and upon notice to the applicable courts, up to five vacant judgeships shall be allocated from superior courts with more authorized judgeships than their assessed judicial need to superior courts with fewer authorized judgeships t

In [26]:
# К примеру был какой то текст, мы его затокенизировали

input_ids = tokenizer.encode(
    text_example,
    return_tensors="pt",
    max_length=1024,
    truncation=True,
    ).to(device)

In [27]:
input_ids.shape # скорее случился trancate так как он больше 1024 размера

torch.Size([1, 1024])

In [28]:
# Реализован в моделях генерации generate

summary_text_ids = model.generate(
    input_ids=input_ids, # наши индексы токенов
    bos_token_id=model.config.bos_token_id, # айди токена начала предложения
    eos_token_id=model.config.eos_token_id, # айди токена конца предложения
    max_length=142,
    min_length=56,
    num_beams=4, # beam search - чтобы не детерминированно выбирали вероятность предсказания слов(а)
)

In [29]:
# то как выглядит сгенерированный текст

summary_text_ids

tensor([[    2,     0,  9089, 15528,   488,  2730,  7396,     5,  9082,     7,
          1045,     8,  1391,    92,  4674,  7903, 22918,     7, 17966,  7668,
             9,     5,   886,  5879,     4, 50118,   713,  1087,    74, 25915,
            62,     7,   292, 11042,  4674,  7903,    31, 10295,  4354,    19,
            55,  8672,  4674,  7903,    87,    49, 11852,  8252,   240,     7,
         10295,  4354,    14,    33,  4163,  8672,  4674,     2]],
       device='cuda:0')

In [30]:
# декодируем индексы в слова, и пропускаем id спец токенов

decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

In [31]:
# Сократили наш текст с 7497 до 332 слов

en(decoded_text), len(text_example)

(332, 7497)

In [32]:
# Посмотрим результат суммаризации

decoded_text

'Existing law authorizes the Legislature to create and fund new judgeships pursuant to specified provisions of the California Constitution.\nThis bill would allocate up to five vacant judgeships from superior courts with more authorized judgeships than their assessed judicial need to superior courts that have fewer authorized judges'

In [33]:
# Теперь сгенерируем предсказания на всем тестовом множестве

summaries = []

for text in tqdm(billsum["test"]['text']):
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        ).to(device)

    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        max_length=142,
        min_length=56,
        num_beams=4,
    )

    decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
    summaries.append(decoded_text)

  0%|          | 0/124 [00:00<?, ?it/s]

## Считаем качество

### ROUGE

In [34]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=4d568de0be1737eca3fa42aa8b7a333fcbdb2992d6e8efed138dda42040f4416
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [35]:
# Обертка над метриками evaluate! В нее загрузим метрику rouge - доля пересечений n-gramm предсказания и исходного текста

rouge = evaluate.load('rouge')

In [36]:
# расчитываем с помощью compute наши метрики

%%time
results = rouge.compute(
        predictions=summaries, # предсказания (суммаризация)
        references=billsum["test"]['summary'] # исходный текст
    )

CPU times: user 3.68 s, sys: 16.2 ms, total: 3.7 s
Wall time: 3.85 s


In [37]:
results

{'rouge1': 0.35032156851195456,
 'rouge2': 0.20044501748396157,
 'rougeL': 0.24041420681074968,
 'rougeLsum': 0.3063602103434004}